In [5]:
%%writefile mapper.py

from __future__ import division
import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')
stop_words = open("stop_words_en.txt").read()

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    words = [word.lower() for word in words if word not in stop_words]
    total_words = len(words)
    aggr_words = Counter(words)
    for word, amount in aggr_words.items():
        tf = amount / total_words
        print "%s\t%s\t%f\t%d" % (word, article_id, tf, 1)


Overwriting mapper.py


In [6]:
%%writefile reducer.py
from __future__ import division
import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')


words_article_sum = 0
current_key = ""

for line in sys.stdin:
    try:
        key,articles,tfs,count = line.strip().split('\t', 3)
        articles = articles.split(",")
        tfs = tfs.split(",")
        count = int(count)
    except ValueError as e:
        continue
    
    if current_key != key:
        if current_key:
            #print(current_key)
            print "%s\t%s\t%s\t%d" % ("".join(current_key), ",".join(current_articles), ",".join(current_tfs), words_article_sum)
        current_key = key
        current_articles = []
        current_tfs = []
        words_article_sum = 0
    words_article_sum += count
    current_articles += articles
    current_tfs += tfs

if current_key:
    print "%s\t%s\t%s\t%d" % ("".join(current_key), ",".join(current_articles), ",".join(current_tfs), words_article_sum)

Overwriting reducer.py


In [7]:
%%writefile parser.py
from __future__ import division
import sys
from math import log


words_article_sum = 0
current_key = ""

for line in sys.stdin:
    try:
        word,articles,tfs,count = line.strip().split('\t', 3)
        articles = articles.split(",")
        tfs = tfs.split(",")
        count = int(count)
    except ValueError as e:
        continue
    if word == "labor":
        for key, value in enumerate(articles):
            if articles[key] == "12":
                print(tfs[key])
                print((1 / log(1 + count))
                print(count)
                tf_idf = float(tfs[key]) * (1 / log(1 + count))
                print(tf_idf)

Overwriting parser.py


In [ ]:
%%bash

OUT_DIR="tfidf_result"$(date +"%s%3N")
NUM_REDUCERS=8
OUT_DIR2="tfidf_final_result"$(date +"%s%3N")


hdfs dfs -cat /datasets/stop_words_en.txt > stop_words_en.txt

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming tfidf" \
    -D mapreduce.job.reduces=1 \
    -files parser.py \
    -reducer "python parser.py" \
    -mapper /bin/cat \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null
    
hdfs dfs -cat ${OUT_DIR2}/part-00000